# Welcome to GenStateChain!

GenStateChain is built off of qutip, and is designed to calculate quantum mechanics with chains of "states", which from now on, I will call atoms :)

Each atom can have an arbitrary number of states, and GenStateChain supports each atom having a different number of states. We will first discuss how to form states and then operators. After that, we will calculate dynamics consisting of several "pulses" and calculate fidelities.

## Preliminaries

We import the module and will set the number of states of each atom in the chain. The number of states of each atom is a **module level property** and all functions from the module will make use of this setting. If it is reset, operators and states created by the module before the change will **not** be compatible with states and operators after the change.

In [2]:
import GenStateChain as GSC
GSC.set_n([2,3,2]) # We have created an atom chain with 2,3 and 2 states.

## Specifying a State

The main way we specify a state is to use `GenStateChain.States.specify`. We provide a list with the state of each atom. Note that the atom states are zero-indexed much like everything in python.

In [4]:
state011 = GSC.States.specify([0,1,1]) # Atoms in states 0,1, and 1
state011.full() # The numpy version of the state

array([[0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]])

For convenience, `GenStateChain.States.all_ground` exists to quickly get a state which is the ground state for all atoms

In [6]:
all_ground_state = GSC.States.all_ground()
all_ground_state.full()

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]])

## Specifying Operators

Operators primarily live in `GenStateChain.Operators`. An identity operator can be created with `GenStateChain.Operators.identity`. Note that we can calculate expectation values with `qutip.expect`. The qutip module can be accessed by importing it. `GenStateChain` also provides the library as `GenStateChain.qt`.

In [9]:
print(GSC.Operators.identity())
print(1 == GSC.qt.expect(GSC.Operators.identity(), state011))
print(1 == GSC.qt.expect(GSC.Operators.identity(), all_ground_state))

Quantum object: dims = [[2, 3, 2], [2, 3, 2]], shape = (12, 12), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
True
True


The full list of operators can be found in the documentation, but we will discuss a few useful ones here. The number operator returns the state number for that atom. The operator is constructed with `GenStateChain.Operators.num(idx_list)`, where `idx_list` are the indices for the atoms that should have a number operator. All other atoms experience identity. 

In [14]:
num1 = GSC.Operators.num([1]) # Number operator on atom 1
print(1 == GSC.qt.expect(num1, state011)) # atom 1 is in state 1
print(0 == GSC.qt.expect(num1, all_ground_state)) # atom 1 is in state 0

state021 = GSC.States.specify([0,2,1])
num12 = GSC.Operators.num([1,2]) # tensor product of the number operators on atoms 1 and 2
num01 = GSC.Operators.num([0,1]) # tensor product of the number operators on atoms 0 and 1

print(2 == GSC.qt.expect(num12, state021))
print(0 == GSC.qt.expect(num01, state021))

True
True
True
True


Another useful operator is the projection operator in `GenStateChain.Operators.proj(state_list)`. The `state_list` here is a state for each atom to be projected into. A `-1` indicates identity on this atom.

In [20]:
proj011 = GSC.Operators.proj([0,1,1]) # Projection operator into state 011
print(1 == GSC.qt.expect(proj011, state011)) # should be 1
print(0 == GSC.qt.expect(proj011, state021)) # should be 0

proj0x1 = GSC.Operators.proj([0, -1, 1])
# This operator is state agnostic on the middle site so it should be 1 for both.
print(1 == GSC.qt.expect(proj0x1, state011))
print(1 == GSC.qt.expect(proj0x1, state021))

True
True
True
True


Paulis on a particular "qubit" subspace are also quite useful. For this, we have `GenStateChain.Operators.sigmax(idx_list, states, bUnitary=0)` (Also see `GenStateChain.Operators.sigmay` and `GenStateChain.Operators.sigmaz`). Here, `idx` functions similarly to `idx_list` of the number operator. It allows for the construction of products of "Pauli" operators. `states` is a list of lists, where each element is the particular qubit subspace for the operator. `bUnitary` controls what happens to other states of that atom. If it is `1`, then identity is performed on the other states (useful if this operator is supposed to represent unitary evolution). If it is `0`, then it annihilates other states (useful if this operator is supposed to be part of a Hamiltonian).

In [28]:
swap_op = GSC.Operators.sigmax([1], [[0,2]]) # Swaps states 0 and 2 on atom 1

state001 = GSC.States.specify([0,0,1])
print(state001.dag() * swap_op * state021) # should be 1
print(0 == GSC.qt.expect(swap_op, state021))
# We check with state 1 on atom 1
print(swap_op * state011)

# Compare with unitary version
swap_op_unitary = GSC.Operators.sigmax([1], [[0,2]], bUnitary=1)
print(1 == GSC.qt.expect(swap_op_unitary, state011))

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[1.]]
True
Quantum object: dims = [[2, 3, 2], [1, 1, 1]], shape = (12, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
True
